In [1]:
using Pkg
Pkg.activate("/mnt/dv/wid/projects4/SolisLemus-network-merging/")
Pkg.instantiate()

using InPhyNet, PhyloNetworks
include("/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/simulation-scripts/helpers/helpers.jl")
cd("/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences")

  Activating project at `/mnt/dv/wid/projects4/SolisLemus-network-merging`


In [26]:
# "Instance" variables
ntaxa = 500
replicatenum = 1
ngt = 100
seq_len = 500
ils_level = "med"
m = 20
dmethod = "AGIC"

true_net = load_true_net_ils_adjusted_level1(ntaxa, replicatenum, ils_level)
# for e in true_net.edge
#     if e.length == -1. e.length = 0.473 end
# end

HybridNetwork, Rooted Network
1048 edges
1033 nodes: 501 tips, 16 hybrid nodes, 516 internal tree nodes.
tip labels: t3, t4, t22, t23, ...
(((((((t3:15.759,t4:15.759):4.393,(((((t22:13.338,t23:13.338):0.228,t18:13.566):1.843,t5:15.41):3.417,(((t13:13.702,(t17:13.627,(t19:13.546,t20:13.546):0.082):0.074):1.562,(t12:13.717)#H1:1.546::0.53):1.634,(((t8:14.049,t9:14.049):2.018,(t10:13.742,t11:13.742):2.325):0.112,(t6:14.252,(t16:13.67,((t24:13.173,t25:13.173):0.322,t21:13.495):0.175):0.582):1.927):0.718):1.929):0.725,((t2:16.74,(((t14:13.685,t15:13.685):0.032,#H1:0.0::0.47):0.517,t7:14.234):2.506):1.034,t1:17.774):1.778):0.6):2.514,(((((((t233:10.505,(t235:10.238,((t241:9.987,t242:9.987):0.001)#H9:0.249::0.554):0.268):0.248,(t243:9.878,t244:9.878):0.875):0.497,t229:11.251):1.583,(t227:11.965,(t228:11.774,(t230:10.937,t231:10.937):0.837):0.191):0.869):0.108,(((t236:10.094,t237:10.094):1.946)#H10:0.28::0.625,((((((((t238:10.019)#H11:0.0::0.548,t239:10.019):0.632,((t249:9.735,t250:9.735):0.75

In [27]:
# File paths
data_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/temp_data/"
checkpoint_dir = joinpath(pwd(), "temp_data")
if !isdir(checkpoint_dir) mkdir(checkpoint_dir) end

truegt_file = joinpath(checkpoint_dir, "truegt_n$(ntaxa)_$(replicatenum)_$(ngt)_$(ils_level).treefile")
seq_file_prefix = joinpath(checkpoint_dir, "seqfile_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level).phy")
estgt_file = joinpath(checkpoint_dir, "estgt_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level).treefile")
net_file = joinpath(checkpoint_dir, "estnets_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod).netfile")

if !isfile(estgt_file) touch(estgt_file) end

# Estimated gene trees

In [28]:
seed = parse(Int64, "$(true_net.numTaxa)42$(true_net.numHybrids)42$(replicatenum)")

true_gts::Vector{HybridNetwork} = simulate_gene_trees(truegt_file, ngt, seed)
pmap(
    (i, gt) -> est_gt_from_true_gt(gt, "$(seq_file_prefix)_$(i)", "$(estgt_file)_$(i)", data_dir, i),
    1:length(true_gts), true_gts
)
true_gts = []
@everywhere GC.gc()

[TRUE GT] Loaded from file.
[SEQ] Sequences already simulated for #1.
[IQ-TREE] Results already exist for #1.
[SEQ] Sequences already simulated for #2.
[IQ-TREE] Results already exist for #2.
[SEQ] Sequences already simulated for #3.
[IQ-TREE] Results already exist for #3.
[SEQ] Sequences already simulated for #4.
[IQ-TREE] Results already exist for #4.
[SEQ] Sequences already simulated for #5.
[IQ-TREE] Results already exist for #5.
[SEQ] Sequences already simulated for #6.
[IQ-TREE] Results already exist for #6.
[SEQ] Sequences already simulated for #7.
[IQ-TREE] Results already exist for #7.
[SEQ] Sequences already simulated for #8.
[IQ-TREE] Results already exist for #8.
[SEQ] Sequences already simulated for #9.
[IQ-TREE] Results already exist for #9.
[SEQ] Sequences already simulated for #10.
[IQ-TREE] Results already exist for #10.
[SEQ] Sequences already simulated for #11.
[IQ-TREE] Results already exist for #11.
[SEQ] Sequences already simulated for #12.
[IQ-TREE] Results alrea

In [32]:
# Consolidate all estimated gene trees into a single file
est_gts = Array{HybridNetwork}(undef, ngt)
for i = 1:ngt
    est_gts[i] = readTopology("$(estgt_file)_$(i)")
end

@everywhere GC.gc()

# Subset decomposition

In [33]:
_, _, nj_tre = estimate_nj_tree(est_gts)
subsets = sateIdecomp(nj_tre, m)

[NJ] Calculating AGIC.
[NJ] Estimating NJ tree.


40-element Vector{Vector{String}}:
 ["t329", "t334", "t341", "t342", "t333", "t336", "t337", "t338", "t349", "t350"]
 ["t326", "t327", "t328", "t335", "t345", "t346", "t339", "t340", "t347", "t348", "t330", "t331", "t332", "t343", "t344"]
 ["t402", "t403", "t404", "t412", "t413", "t418", "t419", "t422", "t423", "t411", "t420", "t421"]
 ["t401", "t405", "t406", "t407", "t408", "t416", "t417", "t409", "t410", "t414", "t415", "t424", "t425"]
 ["t176", "t183", "t184", "t187", "t188", "t180", "t181", "t191", "t192", "t195", "t196"]
 ["t178", "t193", "t194", "t177", "t179", "t182", "t197", "t198", "t185", "t189", "t190", "t199", "t200", "t186"]
 ["t226", "t234", "t239", "t249", "t250", "t245", "t246", "t238", "t247", "t248", "t236", "t237"]
 ["t229", "t233", "t235", "t243", "t244", "t242", "t241", "t232", "t240", "t227", "t228", "t230", "t231"]
 ["t252", "t268", "t269", "t270", "t271", "t255", "t256", "t262", "t263", "t259", "t266", "t272", "t273"]
 ["t251", "t261", "t267", "t274", "t275", "

# SNaQ inference

In [17]:
df_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/CFs/"
nj_dir = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/NJs/"
nruns = 10
GC.gc()

for (i, subset_taxa) in enumerate(subsets)
    output_file = "$(net_file)_$(i)"
    runtime_file = "$(output_file).runtime"
    output_net_file = "$(output_file).netfile"
    
    if isfile(output_net_file) && isfile(runtime_file)
        log("SNaQ $(i)", "Already inferred.")
        continue
    end

    temp_gts = Array{HybridNetwork}(undef, length(est_gts))
    for i = 1:length(est_gts)
        temp_gts[i] = pruneTruthFromDecomp(est_gts[i], subset_taxa)
    end

    # 1. Quartets
    q, t = countquartetsintrees(temp_gts)
    df = silently() do
        readTableCF(writeTableCF(q, t))
    end
    CSV.write(joinpath(df_dir, "df_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).csv"), writeTableCF(df))

    # 2. Starting trees
    init_tree = pruneTruthFromDecomp(nj_tre, subset_taxa)
    tre_out = joinpath(nj_dir, "nj_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).tre")
    open(tre_out, "w+") do f
        write(f, writeTopology(init_tree))
    end

    # 3. Write info to condor input table
    tab_file = "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/condor/inputs.tab"
    open(tab_file, "a") do f
        write(f, "$(ntaxa),$(replicatenum),$(ngt),$(seq_len),$(ils_level),$(m),$(i)\n")
    end

    # 4. Constraints inferred in Condor
end
@everywhere GC.gc()

Reading in trees, looking at 210 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 1365 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 495 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 715 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 330 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in trees, looking at 1001 quartets in each...
0+--------------------------------------------------+100%
  **************************************************
Reading in tre

# InPhyNet

In [37]:
# Load estimated data
# est_constraints = Array{HybridNetwork}(undef, length(subsets))
est_constraints = Vector{HybridNetwork}([])
for i = 1:length(subsets)
    net_file = joinpath(
        "/mnt/dv/wid/projects4/SolisLemus-network-merging/simulation-study/from-sequences/snaq_nets/",
        "snaq_n$(ntaxa)_$(replicatenum)_$(ngt)_$(seq_len)_$(ils_level)_$(m)_$(dmethod)_sub$(i).out"
    )
    if !isfile(net_file) continue end

    net = split(readlines(net_file)[1], " -Ploglik")[1]
    push!(est_constraints, readTopology(net))
    # est_constraints[i] = readTopology(net)
end

# est_constraint_runtimes = Vector{Float64}(
#     [parse(Float64, readlines("$(net_file)_$(i).runtime")[1]) for i=1:length(subsets)]
# )

In [38]:
est_constraints

38-element Vector{HybridNetwork}:
 HybridNetwork, Rooted Network
17 edges
18 nodes: 10 tips, 0 hybrid nodes, 8 internal tree nodes.
tip labels: t341, t342, t334, t329, ...
(t341,t342,(t334,(t329,(t333,(((t336,t337):0.037,t338):0.141,(t349,t350):0.6):0.015):0.94):0.258):0.384);

 HybridNetwork, Rooted Network
27 edges
28 nodes: 15 tips, 0 hybrid nodes, 13 internal tree nodes.
tip labels: t327, t328, t326, t343, ...
(t327,t328,(t326,(((t343,t344):0.949,(t330,(t331,t332):0.157):0.207):2.523,((t335,(t345,t346):0.365):3.716,(t339,(t340,(t347,t348):0.246):0.216):3.994):0.229):0.736):2.483);

 HybridNetwork, Rooted Network
24 edges
24 nodes: 12 tips, 1 hybrid nodes, 11 internal tree nodes.
tip labels: t412, t413, t418, t419, ...
(t412,t413,((t418,(t419)#H13:::0.764):0.576,((t404,(t403,(t402,(t411,(t420,t421):0.425):4.729):0.299):0.724):0.589,(t423,(t422,#H13:::0.236):0.329):0.826):0.17):0.078);

 HybridNetwork, Rooted Network
23 edges
24 nodes: 13 tips, 0 hybrid nodes, 11 internal tree nodes.

In [39]:
est_gts = Array{HybridNetwork}(undef, ngt)
for i = 1:ngt
    est_gts[i] = readTopology("$(estgt_file)_$(i)")
end
est_D, est_namelist = calculateAGID(est_gts)

([0.0 2.82 … 18.74 8.96; 2.82 0.0 … 18.8 9.02; … ; 18.74 18.8 … 0.0 12.22; 8.96 9.02 … 12.22 0.0], ["t115", "t120", "t121", "t111", "t112", "t109", "t102", "t105", "t108", "t113"  …  "t88", "t99", "t87", "t100", "t77", "t80", "t81", "t94", "t89", "OUTGROUP"])

In [45]:
for c in est_constraints
    rootatnode!(c, getchildren(c.node[c.root])[1])
end

In [46]:
est_constraints[12]

HybridNetwork, Rooted Network
26 edges
27 nodes: 14 tips, 0 hybrid nodes, 13 internal tree nodes.
tip labels: t99, t100, t88, t86, ...
(t99,(t100,(t88,((t86,t87):0.184,((t97,t98):4.418,(t77,((t96,((t89,t94):0.774,(t80,t81):0.09):0.517):1.119,t76):0.089):0.485):2.452):0.344):0.502));


In [47]:
inphynet_time = @elapsed mnet = netnj(est_D, est_constraints, est_namelist)

4.774506303

In [54]:
rootatnode!(mnet, "OUTGROUP")
rootatnode!(true_net, "OUTGROUP")
hardwiredClusterDistance(mnet, true_net, true)
mnet.numHybrids, true_net.numHybrids

(9, 16)

In [ ]:
save_estimated_gts_results(
    "$(ntaxa)", true_net, replicatenum, ngt, ils_level,
    m, dmethod, seq_len, mnet, est_constraints,
    est_gts, est_constraint_runtimes, inphynet_time
)